## Build (or load) a model


In [1]:
import pins
import vetiver

import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

api_key = os.getenv("API_KEY")
rsc_url = os.getenv("RSC_URL")

board = pins.board_rsconnect(api_key=api_key, server_url=rsc_url, allow_pickle_read=True)

In [2]:
v = vetiver.VetiverModel.from_pin(
    board, 
    "isabel.zimmerman/ridership", 
    version="60193"
)

Bring in some new data that lives on Connect.

In [3]:
chicago = board.pin_read("isabel.zimmerman/new_chicago")
original_chicago = chicago.iloc[:100, :].copy()

In [4]:
original_chicago["preds"] = v.model.predict(
    original_chicago.drop(columns=["ridership", "date"])
    )

## Compute metrics

Let's say we collect new data on fuel efficiency in cars and we want to monitor the performance of our model over time. We can compute multiple metrics at once over a certain time aggregation.


In [5]:
from sklearn import metrics
from datetime import timedelta

metric_set = [metrics.mean_absolute_error, metrics.mean_squared_error]
td = timedelta(weeks = 1)

In [6]:
original_metrics = vetiver.compute_metrics(data = original_chicago, 
                    date_var="date", 
                    period = td, 
                    metric_set=metric_set, 
                    truth="ridership", 
                    estimate="preds")
original_metrics

,index,n,metric,estimate
0,2014-10-01,7,mean_absolute_error,0.737403
1,2014-10-01,7,mean_squared_error,1.160815
2,2014-10-08,7,mean_absolute_error,1.349887
3,2014-10-08,7,mean_squared_error,2.743981
4,2014-10-15,7,mean_absolute_error,0.734174
5,2014-10-15,7,mean_squared_error,0.699126
6,2014-10-22,7,mean_absolute_error,1.131571
7,2014-10-22,7,mean_squared_error,2.773453
8,2014-10-29,7,mean_absolute_error,1.312606
9,2014-10-29,7,mean_squared_error,3.206079


## Pin metrics

The first time you pin monitoring metrics, you can write to a board as normal. 


In [7]:
my_board = pins.board_folder(path=".")
my_board.pin_write(original_metrics, "metrics", type = "csv")

Writing pin:
Name: 'metrics'
Version: 20220810T132526Z-9b981


Meta(title='metrics: a pinned 30 x 4 DataFrame', description=None, created='20220810T132526Z', pin_hash='9b9813939e75503c', file='metrics.csv', file_size=1556, type='csv', api_version=1, version=Version(created=datetime.datetime(2022, 8, 10, 13, 25, 26, 93420), hash='9b9813939e75503c'), name='metrics', user={})

However, when adding new metrics measurements to your pin as you continue to gather new data and monitor, you may have dates that overlap with those already in the pin, depending on your monitoring strategy. You can choose how to handle overlapping dates with the `overwrite` argument.


In [8]:
#| output: false
# dates overlap with existing metrics:
new_chicago = chicago.iloc[75:, :].copy()
new_chicago["preds"] = v.model.predict(
    new_chicago.drop(columns=["date", "ridership"])
)

new_metrics = vetiver.compute_metrics(data = new_chicago, 
                    date_var="date", 
                    period = td, 
                    metric_set=metric_set, 
                    truth="ridership", 
                    estimate="preds")                    

In [9]:
vetiver.pin_metrics(
    my_board, 
    new_metrics, 
    "metrics", 
    overwrite = True
)

Writing pin:
Name: 'metrics'
Version: 20220810T132556Z-9871c


,index,n,metric,estimate
0,2014-10-01,7,mean_absolute_error,0.737403
1,2014-10-01,7,mean_squared_error,1.160815
2,2014-10-08,7,mean_absolute_error,1.349887
3,2014-10-08,7,mean_squared_error,2.743981
4,2014-10-15,7,mean_absolute_error,0.734174
...,...,...,...,...
203,2016-08-08,7,mean_squared_error,1.402398
204,2016-08-15,7,mean_absolute_error,0.922150
205,2016-08-15,7,mean_squared_error,1.195525
206,2016-08-22,7,mean_absolute_error,0.946992


## Plot metrics

You can visualize your set of computed metrics and your model's performance.


In [10]:
m = my_board.pin_read("metrics", version="20220810T132556Z-9871c")

In [11]:
monitor_plot = vetiver.plot_metrics(m)
monitor_plot.update_yaxes(matches=None)
monitor_plot.show()